In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import keras

In [3]:
df=pd.read_csv('text.csv')

In [4]:
df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [5]:
df.shape

(416809, 3)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  416809 non-null  int64 
 1   text        416809 non-null  object
 2   label       416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ MB


In [7]:
df.columns

Index(['Unnamed: 0', 'text', 'label'], dtype='object')

In [8]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [9]:
df.describe()

,label
count,416809.000000
mean,1.554271
std,1.490453
min,0.000000
25%,0.000000
50%,1.000000
75%,3.000000
max,5.000000


In [10]:
df.head()

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [11]:
emotion_dict = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

In [12]:
# df['length'] = df['text'].apply(len)

In [13]:
print(f"Rating value count: \n{df['label'].value_counts()}")

Rating value count: 
label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64


In [14]:
print(f"Label value count - percentage distribution: \n{round(df['label'].value_counts()/df.shape[0]*100,2)}")

Label value count - percentage distribution: 
label
1    33.84
0    29.07
3    13.75
4    11.45
2     8.29
5     3.59
Name: count, dtype: float64


In [15]:
data2 = df.groupby('label').apply(lambda x: x.sample(14000)).reset_index(drop=True)

/var/folders/8s/589js3cj3k5gf6dtzp4t6v7r0000gn/T/ipykernel_59761/964725730.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data2 = df.groupby('label').apply(lambda x: x.sample(14000)).reset_index(drop=True)


In [16]:
print(f"Rating value count: \n{data2['label'].value_counts()}")

Rating value count: 
label
0    14000
1    14000
2    14000
3    14000
4    14000
5    14000
Name: count, dtype: int64


In [17]:
data2

,text,label
0,i feel like the past two and half years were a...,0
1,i guess this is how i do feel lost beaten,0
2,id had a friend from my hometown to stay for t...,0
3,i have ever met and he truly feels hurt by wha...,0
4,i sure hope this isnt true because it would me...,0
...,...,...
83995,i woke up feeling kind of overwhelmed,5
83996,i have to throw it in here because the feeling...,5
83997,i feel and it shocked him beyond belief so,5
83998,i had never felt but needed to feel i stood th...,5


In [18]:
X=data2.drop('label',axis=1)

In [19]:
y=data2['label']

In [20]:
X.shape,y.shape

((84000, 1), (84000,))

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.layers import Dense

In [22]:
voc_size=10000

In [23]:
import nltk
import re
from nltk.corpus import stopwords

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pranayyb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [26]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [27]:
corpus=[]
for i in range(0, len(X)):
    text = re.sub('[^a-zA-Z]', ' ', data2['text'][i])
    text = text.lower()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    corpus.append(text)

In [28]:
len(corpus)

84000

In [29]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
len(onehot_repr)

84000

In [30]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)

In [31]:
embedded_docs.shape

(84000, 20)

In [32]:
from tensorflow.keras.layers import Dropout

In [33]:
embedding_vector_features = 20
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(500, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(500, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(200))
model.add(Dense(6, activation='softmax'))  
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-08-20 14:34:23.813324: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-08-20 14:34:23.813343: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-20 14:34:23.813346: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-20 14:34:23.813504: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-20 14:34:23.813514: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [34]:
len(embedded_docs),y.shape

(84000, (84000,))

In [35]:
embedded_docs[83564]

array([6349,  508, 9310, 1619, 5527, 2049,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [36]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=29)

In [38]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20)

Epoch 1/20


2024-08-20 14:34:24.800852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


2100/2100 ━━━━━━━━━━━━━━━━━━━━ 335s 157ms/step - accuracy: 0.5548 - loss: 1.0896 - val_accuracy: 0.8769 - val_loss: 0.3875
Epoch 2/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 369s 176ms/step - accuracy: 0.9047 - loss: 0.2966 - val_accuracy: 0.9107 - val_loss: 0.2598
Epoch 3/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 562s 267ms/step - accuracy: 0.9208 - loss: 0.2204 - val_accuracy: 0.9192 - val_loss: 0.2294
Epoch 4/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 424s 202ms/step - accuracy: 0.9275 - loss: 0.1951 - val_accuracy: 0.9190 - val_loss: 0.2332
Epoch 5/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 719s 343ms/step - accuracy: 0.9306 - loss: 0.1767 - val_accuracy: 0.9192 - val_loss: 0.2279
Epoch 6/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 787s 375ms/step - accuracy: 0.9346 - loss: 0.1656 - val_accuracy: 0.9210 - val_loss: 0.2348
Epoch 7/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 348s 166ms/step - accuracy: 0.9386 - loss: 0.1557 - val_accuracy: 0.9189 - val_loss: 0.2333
Epoch 8/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 362s 172ms/step - accuracy: 0.9

In [39]:
y_pred=model.predict(X_test)

525/525 ━━━━━━━━━━━━━━━━━━━━ 32s 60ms/step


In [40]:
y_pred[0]

array([3.7318672e-04, 3.0538175e-04, 7.7527002e-05, 4.2384446e-01,
       5.7429683e-01, 1.1026814e-03], dtype=float32)

In [41]:
y_pred_classes = np.round(y_pred).astype(int)

In [42]:
y_pred_classes

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [43]:
y_test

array([3, 3, 2, ..., 0, 1, 3])

In [44]:
from sklearn.metrics import accuracy_score

if len(y_pred_classes.shape) > 1 and y_pred_classes.shape[1] > 1:
    y_pred_classes = np.argmax(y_pred_classes, axis=1)

if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Accuracy: {accuracy}')


Accuracy: 0.8976190476190476


In [149]:
def predict(text,model):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text if word not in stopwords.words('english')]
    text = ' '.join(text)
    onehot_repr = [one_hot(text, voc_size)]
    embedded_text = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
    X_final = np.array(embedded_text)
    y_pred = model.predict(X_final)
    y_pred_classes = np.argmax(y_pred, axis=1)
    emotion=emotion_dict[y_pred_classes[0]]
    return emotion

In [151]:
data="his situation is just making me so angry!"

In [155]:
predict(data,model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


'anger'

In [157]:
import pickle

In [159]:
with open('emotion.pkl', 'wb') as file:
    pickle.dump(model, file)

In [161]:
with open('emotion.pkl', 'rb') as file:
    loaded_model = pickle.load(file)